<a href="https://colab.research.google.com/github/fginter/ainl_2020_tutorial/blob/main/parser_tnpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turku Neural Parser Pipeline - Python module version

* This is a basic tutorial for running the parser pipeline under Google Colab
* Many new properties added for the AINL2020 tutorial
  * Parser changed to Udify (87->91% LAS improvement)
  * Code restructured so as to be importable as a module and can be installed using pip
  * No longer depends on Tensorflow
* Makes it possible for anyone to run the parser with GPU acceleration


# Install

* Pre-built wheel, not yet in PyPi
* Only Finnish and English models in this tutorial, more coming!

## Install the parser package (takes its time)

`pip3 install http://dl.turkunlp.org/turku-parser-models/turku_neural_parser-0.2-py3-none-any.whl`

## Download and unpack the model

`wget http://dl.turkunlp.org/turku-parser-models/models_fi_tdt.tar.gz ; tar zxvf models_fi_tdt.tar.gz`

...and you are good to go!

In [ ]:
!pip3 install http://dl.turkunlp.org/turku-parser-models/turku_neural_parser-0.2-py3-none-any.whl

In [ ]:
!wget  http://dl.turkunlp.org/turku-parser-models/models_fi_tdt.tar.gz ; tar zxvf models_fi_tdt.tar.gz

# Running the parser

* Every model can specify many processing pipelines
* These are in `modeldir/pipelines.yaml`
* `parse_plaintext`is the default

In [ ]:
from tnparser.pipeline import read_pipelines, Pipeline

available_pipelines=read_pipelines("models_fi_tdt/pipelines.yaml")
print(list(available_pipelines.keys()))


In [ ]:
p=Pipeline(available_pipelines["parse_plaintext"])
parsed=p.parse("Minulla on ruskea koira! Se haukkuu ja juoksee. Voi että!")

In [ ]:
print(parsed)

# GPU mode

* The pipeline runs by default in CPU mode
* Needs to be told to run in GPU
* This is a bit tricky right now but not impossible
* Note: if you now switch the Runtime into GPU, you need to re-run the pip install


In [ ]:
#I do realize this ain't good! :)
import types
extra_args=types.SimpleNamespace()
extra_args.__dict__["udify_mod.device"]="0" #simulates someone giving a --device 0 parameter to Udify
extra_args.__dict__["lemmatizer_mod.device"]="0" 

p=Pipeline(available_pipelines["parse_plaintext"],extra_args)
parsed=p.parse("Minulla on ruskea koira! Se haukkuu ja juoksee. Voi että!")
print("Parsed has this many lines:",len(parsed.split("\n")))


In [ ]:
#Since we are on a GPU, we can try to push through quite a bit more of data
parsed=p.parse("Minulla on ruskea koira! Se haukkuu ja juoksee. Voi että! "*200) #takes forever on CPU, finishes in few seconds on GPU
print("Parsed has this many lines:",len(parsed.split("\n")))

# Process the output

* The output of the pipeline run is a conll-u string
* You can parse it in any number of ways
* This is my preferred:

In [ ]:
ID,FORM,LEMMA,UPOS,XPOS,FEAT,HEAD,DEPREL,DEPS,MISC=range(10) #the 10 columns

def read_conll(inp,max_sent=0,drop_tokens=True,drop_nulls=True):
    """
    inp: list of lines or an open file
    max_sent: 0 for all, >0 to limit
    drop_tokens: ignore multiword token lines
    drop_nulls: ignore null nodes in enhanced dependencies

    Yields lines of the parse and comments
    """

    comments=[]
    sent=[]
    yielded=0
    for line in inp:
        line=line.rstrip("\n")
        if line.startswith("#"):
            comments.append(line)
        elif not line:
            if sent:
                yield sent,comments
                yielded+=1
                if max_sent>0 and yielded==max_sent:
                    break
                sent,comments=[],[]
        else:
            cols=line.split("\t")
            if drop_tokens and "-" in cols[ID]:
                continue
            if drop_nulls and "." in cols[ID]:
                continue
            sent.append(cols)
    else:
        if sent:
            yield sent,comments

for one_sent,comments in read_conll(parsed.split("\n"),5):
    words=(word_line[FORM] for word_line in one_sent)
    lemmas=(word_line[LEMMA] for word_line in one_sent)
    print(" ".join(words))
    print(" ".join(lemmas))
    print()

# and that's really all there is to it :)
